In [ ]:
###Models to do
# models on cleaned data
# models on averages/trend data
# models on new data (eg. PAV, ladder, ohe venue)
    #make sure to include absolute margin when testing and evaluating these
    #keep track in excel?
# lazy predict

In [1]:
import re
import xgboost as xgb
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)
#import torch.nn as nn
#import touch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import randint
from Gather_AFL_Data import gatherer as gad
#from fdnn import feature_extractor as fex
import skopt
from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score, KFold
from joblib import dump, load

/home/chris/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


2.2.0
2.4.3


In [ ]:
import re
import xgboost as xgb
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)
#import torch.nn as nn
#import touch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import randint
from Gather_AFL_Data import gatherer as gad
#from fdnn import feature_extractor as fex
import skopt
from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Activation
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score, KFold

In [2]:
#get headers
#feed this into a bigger function which specifies the amount of games to go through 
def get_headers(n_games):
    headers = ['Round', 'Home_Team', 'Away_Team', 'Venue']
    example_file = pd.read_csv('Data/Fremantle_clean_stats.csv')
    cl_h = example_file.columns
    cl_h = cl_h[:-5]
    ladder_header = ['Ladder Pos_H', 'Form_H', 'Season Wins_H', 'Season Loss_H', 'Season Draw_H']
    headers = [*headers, *ladder_header]
    j = 1
    while j <= n_games:
        for x in cl_h:
            if 'Match_ID' in x or 'Year' in x:
                continue
            x = 'H_'+ x + ' n-' + str(j)
            headers.append(x)
        j = j + 1
    j = 1
    ladder_header = ['Ladder Pos_A', 'Form_A', 'Season Wins_A', 'Season Loss_A', 'Season Draw_A']
    headers = [*headers, *ladder_header]
    while j <= n_games:
        for x in cl_h:
            if 'Match_ID' in x or 'Year' in x:
                continue
            x = 'A_'+ x + ' n-' + str(j)
            headers.append(x)
        j = j + 1
    return headers


In [3]:
def clean_headers(h):
    headers = []
    for x in h:
        if '<' in x or '>' in x:
            x = x.replace('<',"lt_")
            x = x.replace('>', "gt_")
            #print(x)
        headers.append(str(x))
    return headers

In [4]:
def generate_categorical_headers(h):
    cat_var = ['Round', 'Home_Team', 'Away_Team']
    skip = 0
    for x in h:
        if 'Round' in x:
            if (skip == 0):
                skip = 1
                continue
            cat_var.append(x)
            #print(x)
        elif 'Team_against_ID' in x:
            #print(x)
            cat_var.append(x)
        elif 'Venue' in x:
            cat_var.append(x)
    return cat_var

In [5]:
#one hot encode data and transform the X_data
#first redo, find the categorial variables
def ohe_data(x_data, enc, flag,cat_var):
    #data has not been previously one hot encoded
    if (flag == 0):
        #get columns with categorical data and drop from main DF
        categorical_data = x_data[cat_var]
        x_data = x_data.drop(cat_var, axis = 1)
        #define and fit new OHE. Use it on our categorical data by transforming
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        ohe = ohe.fit(categorical_data)
        categorical_data = ohe.transform(categorical_data)
        #get feature names for better labelling
        fn = ohe.get_feature_names(cat_var)
        #make a dataframe with new OHE data and feature names
            #would have been good to have coded it like this for my Masters project...
        categorical_data = pd.DataFrame(categorical_data)
        categorical_data.columns = fn
        #ensure that it won't get cranky about any different indexes(shouldn't be any but just a good check)
        x_data.reset_index(drop=True, inplace=True)
        categorical_data.reset_index(drop=True, inplace=True)
        #concatenate along column axis
        x_data = pd.concat([x_data, categorical_data], axis = 1)
    else:
        #same as above except used already fitted ohe
        categorical_data = x_data[cat_var]
        x_data = x_data.drop(cat_var, axis = 1)
        categorical_data = enc.transform(categorical_data)
        fn = enc.get_feature_names(cat_var)
        categorical_data = pd.DataFrame(categorical_data)
        categorical_data.columns = fn
        x_data.reset_index(drop=True, inplace=True)
        categorical_data.reset_index(drop=True, inplace=True)
        x_data = pd.concat([x_data, categorical_data], axis = 1, ignore_index=True)
        ohe = enc
    return x_data, ohe

In [6]:
#search parameters for best XGB classifier or best XGB regressor

def param_search(x_data, y_label, class_reg):

    def on_step(optim_result):
        """
        Callback meant to view scores after
        each iteration while performing Bayesian
        Optimization in Skopt"""
        score = xgb_bayes_search.best_score_
        print("best score: %s" % score)
        if score >= 0.98:
            print('Interrupting!')
            return True
    X_train, X_test, y_train, y_test = train_test_split(x_data, y_label, test_size=0.1, random_state=157732)
    print("X_train shape: " + str(X_train.shape))
    print("X_test shape: " + str(X_test.shape))
    print("y_train shape: " + str(y_train.shape))
    print("y_test shape: " + str(y_test.shape))
    space ={'learning_rate': Real(0.01, 1.0, 'log-uniform'),
        'min_child_weight': Integer(0, 10),
        'max_depth': Integer(0, 50),
        'max_delta_step': Integer(0, 20),
        'subsample': Real(0.01, 1.0, 'uniform'),
        'colsample_bytree': Real(0.01, 1.0, 'uniform'),
        'colsample_bylevel': Real(0.01, 1.0, 'uniform'),
        'reg_lambda': Real(1e-9, 1000, 'log-uniform'),
        'reg_alpha': Real(1e-9, 1.0, 'log-uniform'),
        'gamma': Real(1e-9, 0.5, 'log-uniform'),
        'min_child_weight': Integer(0, 5),
        'n_estimators': Integer(50, 200),
        'scale_pos_weight': Real(1e-6, 500, 'log-uniform')}
    if(class_reg == 0):
        xgbclass = xgb.XGBClassifier(random_state=27022013)
    else:
        xgbclass = xgb.XGBRegressor(random_state=27022013)
    xgb_bayes_search = BayesSearchCV(xgbclass, space, n_iter=60, # specify how many iterations
                                    scoring=None, n_jobs=1, cv=10, verbose=1, random_state=42, n_points=12,
                                 refit=True)
    kk = np.isinf(X_train)
    if True in kk:
    	print("aaaaaaa")
    kk = np.isinf(y_train)
    if True in kk:
    	print("reeeeeee")
    try:
        xgb_bayes_search.fit(X_train, y_train.values.ravel(), callback = on_step)
    except:
        xgb_bayes_search.fit(X_train, y_train.values.ravel(), callback = on_step)
    print("BEST PARAMS ARE HERE")
    print(xgb_bayes_search.best_params_)
    model = xgb_bayes_search.best_estimator_

    return model

In [7]:
#x = all data
#y = label of whether home team or away team won
#m = xgb classifier for winner prediction
#my = labels of margins
    #try absolute values
#mm = xgb regressor for margin prediction
def eval_xgb_games_margins(x, y, m, my, mm):
    results = []
    error = []
    count = 0
    best_w = m
    high_w = 0
    best_m = mm
    high_m = 100
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    print(x)
    for train,test in cv.split(x,y):
       # print(len(train))
        count = count + 1
        print("Split: " + str(count))
        #comment out fit steps for random forest i guess lol
        prediction = m.fit(x.loc[train],y.loc[train]).predict_proba(x.loc[test])
        margin_pred = mm.fit(x.loc[train], my.loc[train])
        print("variables for auroc curve done. Processing fold accuracy + checking best model")
        y_pred = m.predict(x.loc[test])
        #print(y_pred)
        m_pred = mm.predict(x.loc[test])
        print(m_pred)
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y.loc[test], predictions)
        pcent = accuracy * 100.0
        print("The accuracy of this model is" + str(pcent))
        rmse = sqrt(mean_squared_error(m_pred, my.loc[test]))
        print("The rmse of this model is" + str(rmse))
        results.append(pcent)
        error.append(rmse)
        #change the best model to equal current model
        if(pcent > high_w):
            print("found new best classify")
            best_w = m
            high_w = pcent
        if(rmse < high_m):
            print("found best new margin")
            best_m = mm
            high_m = rmse
    print("Best win percentage split = " +str(high_w))
    print("Best margin rmse = "+str(high_m))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    print("Training Testing Margins: %.2f%% (%.2f%%)" % (np.mean(error), np.std(error)))
    return best_w, best_m

In [8]:
#make this a method param later
#n_games = 5
def run_all_models(n_games):
    #load headers and then subsequently categorical headers
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)
    filename = 'Models/ohe_'+str(n_games)+'_no_2021_games.dat'
    pickle.dump(ohe, open(filename, "wb"))
    #reset headers
    feature_names = x_data.columns

    #loads the ylabel matrix,
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #loads margin as the y_label
    margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')

    print(margin_label.shape)
    print(y_label.shape)
    print(x_data.shape)

    #regex solution which is apparently necessary??
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]

    #optimise XGBoost model

    #for predicting win
    model = param_search(x_data, y_label, 0)
    #for predicting margin
    #margin_label = abs(margin_label)
    margin_model = param_search(x_data, margin_label, 1)

   # margin_label = abs(margin_label)
    print(margin_label)
    best_xgb_clas, best_xgb_reg = eval_xgb_games_margins(x_data, y_label, model, margin_label, margin_model)

    #save
    pickle.dump(best_xgb_clas, open("Models/best_xgb_clas_no2020_"+str(n_games)+"_games.dat", "wb"))
    pickle.dump(best_xgb_reg, open("Models/best_xgb_reg_no2020_"+str(n_games)+"_games.dat", "wb"))

In [9]:
#as one big script to go through n_games
n_games = [1,10]
for n in n_games:
    run_all_models(n)

(1655, 1)
(1655, 1)
(1655, 410)
X_train shape: (1489, 410)
X_test shape: (166, 410)
y_train shape: (1489, 1)
y_test shape: (166, 1)
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   30.5s finished


best score: 0.6306245802552048
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   16.8s finished


best score: 0.6359973136333109
Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

variables for auroc curve done. Processing fold accuracy + checking best model
[ -8.138566    27.7727       3.1968532    4.815761    -8.965088
  21.9728       3.1123433  -14.346012     5.023799     7.3043833
  29.420189    34.510094    10.946198    12.7549715  -10.401021
   7.849125     3.2182987   -1.8269391  -28.837696   -10.071691
  28.704018   -33.778866    42.904053   -20.648512    37.135212
  36.213505   -18.877064    -8.776897    39.858242   -13.219126
 -19.070042   -36.078686    38.619225    35.945953     5.3006706
 -19.620117     6.816554     3.5008764    0.61750054   9.782239
 -28.466652   -44.89548    -14.1875305   27.710497    20.416443
  -0.19192612  10.022134    27.586588    21.284754   -16.9997
  22.20801     29.517994   -29.231775     2.9091904   -7.3772564
  14.907157    -9.6571245   10.171475    23.263262   -15.171487
  46.170723    18.776245     7.090273    11.153395   -13.572049
   6.626371    -2.727374    18.064556    20.685186     9.48696
  16.93787    -11.616109 

/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


variables for auroc curve done. Processing fold accuracy + checking best model
[-17.685524   -31.90797     35.459435    21.667826    21.411037
  30.475224    37.93675     29.420614     9.632371    11.761468
  58.0329      -9.395707    32.84267     13.004001    19.250954
  20.849213    10.761637   -29.127422   -13.21829     20.43735
  14.423061    37.184914    29.780827   -29.014591    -2.6424122
  -2.7350814    7.0317655  -31.738419    21.13003     43.754414
 -16.101562    11.9703245    9.981722    29.482336    25.625093
  -0.09500545  31.922247     1.2908701   14.951483   -26.640053
  44.219116    32.637344   -30.256023     8.326197    54.75777
  11.909005    13.918705     6.774366     0.70858186  34.94073
  19.337183     2.6595073   10.290527    13.048131     4.8352556
  13.951997    22.53334     14.694075    34.16064    -38.139458
 -11.654954   -19.020958    28.271902     7.765479    11.024748
  -0.13493365  -6.753258    10.685881    -4.288646    44.149086
  16.410278   -12.821365  

/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


variables for auroc curve done. Processing fold accuracy + checking best model
[ 35.47012     -9.040147   -10.372275    17.066738    31.739325
  53.11392     -3.1801424   21.98435     16.847982    24.383017
 -25.322334     4.7543325   21.398449     1.3004192    9.812985
 -12.168295   -32.961338    12.248857    31.993717    29.457727
 -14.143319    -2.6745741   32.735657   -35.503616    36.86878
  14.194526    28.935535    17.77362     -1.413845   -15.295385
 -10.175325    32.191456    15.598124    15.181819   -29.02885
   6.0516043  -35.66506     29.77751    -41.21616      7.2174115
  16.24301     -8.864648    -9.815173    31.501528     8.442005
   9.642318    50.036438    -1.9482806   -3.0394595   21.221575
   9.985979   -15.530407    19.608183   -26.159899    27.688047
  -6.796136    28.860937    24.100609    13.282007   -42.688663
  27.35525      1.0007296   54.03902      0.44624084  26.802597
   3.7625368    6.330085    14.924097    16.036095     8.714724
 -24.831888   -17.09923   

# Shit Models do not use

# Nearest Centroid

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from Gather_AFL_Data import gatherer as gad
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.neighbors import NearestCentroid
import re
from numpy import arange
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.naive_bayes import BernoulliNB

In [6]:
def load_data(n_games):
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)

    #reset headers
    feature_names = x_data.columns

    #loads the ylabel matrix,
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #loads margin as the y_label
    margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')

    print(margin_label.shape)
    print(y_label.shape)
    print(x_data.shape)

    #regex solution which is apparently necessary??
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]
    return x_data, y_label

In [ ]:
def params_NC(x_data, y_label):
# define model
    print("optimising hyperparameters")
    model = NearestCentroid()
    # define model evaluation method
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # define grid
    grid = dict()
    grid['shrink_threshold'] = arange(0, 1.01, 0.01)
    grid['metric'] = ['euclidean', 'manhattan']
    # define search
    search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
    # perform the search
    results = search.fit(x_data, y_label.values.ravel())
    # summarize
    print('Mean Accuracy: %.3f' % results.best_score_)
    print('Config: %s' % results.best_params_)
    return results.best_params_

In [ ]:
def eval_NC(x, y, m):
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    results = []
    print(x)
    count = 0
    best_w = m
    high_w = 0
    for train,test in cv.split(x,y):
        count = count + 1
        print("Split: " + str(count))
        x_train = x.loc[train]
        y_train = y.loc[train].values.ravel()
        x_test = x.loc[test]
        y_test = y.loc[test]
        m.fit(x_train,y_train)
        y_pred = m.predict(x_test)
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y_test, predictions)
        pcent = accuracy * 100.0
        results.append(pcent)
        if(pcent > high_w):
            print("found new best classify")
            best_w = m
            high_w = pcent
    print("Best win percentage split = " +str(high_w))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_w

In [ ]:
def run_all_NC(n):
    x_data, y_label = load_data(n)
    best_params = params_NC(x_data, y_label)
    model = NearestCentroid(**best_params)
    best_NC_clas = eval_NC(x_data, y_label, model)
    pickle.dump(best_NC_clas, open("Models/best_NC_clas_no2020"+str(n_games)+'_games.sav', 'wb'))

In [ ]:
n_games = [1,2,3,4,5,6,7,10]
for n in n_games:
    run_all_NC(n)

# BernoulliNB

In [23]:
def params_BNB(x_data, y_label):
# define model
    print("optimising hyperparameters")
    model = BernoulliNB()
    # define model evaluation method
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # define grid
    grid = dict()
    grid['alpha'] = arange(0.01, 1.01, 0.01)
    # define search
    search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=1, verbose=1)
    # perform the search
    results = search.fit(x_data, y_label.values.ravel())
    # summarize
    print('Mean Accuracy: %.3f' % results.best_score_)
    print('Config: %s' % results.best_params_)
    return results.best_params_

In [18]:
def eval_BNB(x, y, m):
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    results = []
    print(x)
    count = 0
    best_w = m
    high_w = 0
    for train,test in cv.split(x,y):
        count = count + 1
        print("Split: " + str(count))
        x_train = x.loc[train]
        y_train = y.loc[train].values.ravel()
        x_test = x.loc[test]
        y_test = y.loc[test]
        m.fit(x_train,y_train)
        y_pred = m.predict(x_test)
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y_test, predictions)
        pcent = accuracy * 100.0
        results.append(pcent)
        if(pcent > high_w):
            print("found new best classify")
            best_w = m
            high_w = pcent
    print("Best win percentage split = " +str(high_w))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_w

In [19]:
def run_all_BNB(n):
    x_data, y_label = load_data(n)
    best_params = params_BNB(x_data, y_label)
    model = BernoulliNB(**best_params)
    best_BNB_clas = eval_BNB(x_data, y_label, model)
    pickle.dump(best_BNB_clas, open("Models/best_BNB_clas_no2020"+str(n_games)+'_games.sav', 'wb'))

In [24]:
n_games = [1,2,3,4,5,6,7,10]
for n in n_games:
    run_all_BNB(n)

(1646, 1)
(1646, 1)
(1646, 409)
optimising hyperparameters
Fitting 30 folds for each of 100 candidates, totalling 3000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3000 out of 3000 | elapsed:   56.7s finished


Mean Accuracy: 0.602
Config: {'alpha': 0.89}
      Ladder Pos_H  Form_H  Season Wins_H  Season Loss_H  Season Draw_H  \
0             12.0     0.0            0.0            1.0            0.0   
1              4.0     1.0            1.0            0.0            0.0   
2              2.0     1.0            1.0            0.0            0.0   
3             17.0     0.0            0.0            1.0            0.0   
4              1.0     1.0            1.0            0.0            0.0   
...            ...     ...            ...            ...            ...   
1641           3.0     0.0           16.0            6.0            0.0   
1642           4.0     1.0           15.0            7.0            0.0   
1643           1.0     1.0           17.0            4.0            1.0   
1644           2.0     1.0           17.0            5.0            0.0   
1645           1.0     1.0           17.0            4.0            1.0   

      H_H/A? n-1  H_H/A Win? n-1  H_Points For n-1  H_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3000 out of 3000 | elapsed:  1.4min finished


Mean Accuracy: 0.599
Config: {'alpha': 0.7000000000000001}
      Ladder Pos_H  Form_H  Season Wins_H  Season Loss_H  Season Draw_H  \
0              4.0     2.0            2.0            0.0            0.0   
1             12.0     0.0            0.0            2.0            0.0   
2             18.0     0.0            0.0            2.0            0.0   
3             16.0     0.0            0.0            2.0            0.0   
4             11.0     1.0            1.0            1.0            0.0   
...            ...     ...            ...            ...            ...   
1632           3.0     1.0           16.0            6.0            0.0   
1633           4.0     2.0           15.0            7.0            0.0   
1634           1.0     2.0           17.0            4.0            1.0   
1635           2.0     2.0           17.0            5.0            0.0   
1636           1.0     2.0           17.0            4.0            1.0   

      H_H/A? n-1  H_H/A Win? n-1  H_Poin

...

Mean Accuracy: 0.611
Config: {'alpha': 0.9500000000000001}
      Ladder Pos_H  Form_H  Season Wins_H  Season Loss_H  Season Draw_H  \
0              2.0     6.0            6.0            0.0            0.0   
1              3.0     5.0            5.0            1.0            0.0   
2             14.0     2.0            2.0            4.0            0.0   
3             16.0     1.0            1.0            5.0            0.0   
4              6.0     4.0            4.0            2.0            0.0   
...            ...     ...            ...            ...            ...   
1596           3.0     4.0           16.0            6.0            0.0   
1597           4.0     4.0           15.0            7.0            0.0   
1598           1.0     4.0           17.0            4.0            1.0   
1599           2.0     6.0           17.0            5.0            0.0   
1600           1.0     4.0           17.0            4.0            1.0   

      H_H/A? n-1  H_H/A Win? n-1  H_Poin

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3000 out of 3000 | elapsed:  4.8min finished


Mean Accuracy: 0.608
Config: {'alpha': 1.0}
      Ladder Pos_H  Form_H  Season Wins_H  Season Loss_H  Season Draw_H  \
0             10.0     3.0            3.0            4.0            0.0   
1              2.0     6.0            6.0            1.0            0.0   
2              3.0     6.0            6.0            1.0            0.0   
3             13.0     3.0            3.0            4.0            0.0   
4              5.0     5.0            5.0            2.0            0.0   
...            ...     ...            ...            ...            ...   
1587           3.0     5.0           16.0            6.0            0.0   
1588           4.0     4.0           15.0            7.0            0.0   
1589           1.0     5.0           17.0            4.0            1.0   
1590           2.0     6.0           17.0            5.0            0.0   
1591           1.0     5.0           17.0            4.0            1.0   

      H_H/A? n-1  H_H/A Win? n-1  H_Points For n-1  H_P

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3000 out of 3000 | elapsed:  6.8min finished


Mean Accuracy: 0.603
Config: {'alpha': 0.060000000000000005}
      Ladder Pos_H  Form_H  Season Wins_H  Season Loss_H  Season Draw_H  \
0              5.0     7.0            7.0            3.0            0.0   
1             18.0     0.0            0.0           10.0            0.0   
2              9.0     5.0            5.0            5.0            0.0   
3             13.0     4.0            4.0            6.0            0.0   
4             16.0     2.0            2.0            8.0            0.0   
...            ...     ...            ...            ...            ...   
1560           3.0     7.0           16.0            6.0            0.0   
1561           4.0     7.0           15.0            7.0            0.0   
1562           1.0     6.0           17.0            4.0            1.0   
1563           2.0     9.0           17.0            5.0            0.0   
1564           1.0     6.0           17.0            4.0            1.0   

      H_H/A? n-1  H_H/A Win? n-1  H_Po

# LAZY PREDICT

In [ ]:
import pandas as pd
import numpy as np
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split
from Gather_AFL_Data import gatherer as gad
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import pickle

In [ ]:
def eval_lazy_predict(x, y, my):
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    all_clf_models = []
    all_reg_models = []
    count = 0
    for train,test in cv.split(x,y):
       # print(len(train))
        count = count + 1
        print("Split: " + str(count))
        #classifier
        X_train = x.loc[train]
        X_test = x.loc[test]
        y_train = y.loc[train]
        y_test = y.loc[test]
        my_train = my.loc[train]
        my_test = my.loc[test]
        clf = LazyClassifier(predictions=True)
        reg = LazyRegressor(predictions=True)
        if(count == 1):
            all_clf_models, predictions = clf.fit(X_train, X_test, y_train, y_test)
          #  all_reg_models, reg_predictions = reg.fit(X_train, X_test, my_train, my_test)
            all_clf_models.sort_index(inplace=True)
           # all_reg_models.sort_index(inplace=True)
            #print(all_models)
        else:
            new_clf_models, predictions = clf.fit(X_train, X_test, y_train, y_test)
           # new_reg_models, reg_predictions = reg.fit(X_train, X_test, my_train, my_test)
            new_clf_models.sort_index(inplace=True)
           # new_reg_models.sort_index(inplace=True)
            #print(new_models)
            all_clf_models = all_clf_models.add(new_clf_models)
           # all_reg_models = all_reg_models.add(new_reg_models)
    return all_clf_models#, all_reg_models

In [ ]:
def run_all_lp(n_games):
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)

    #reset headers
    feature_names = x_data.columns

    #loads the ylabel matrix,
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #do margins in a second when this works
    margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')
    margin_label = abs(margin_label)

    clf_lp = eval_lazy_predict(x_data, y_label, margin_label)

    clf_lp.sort_values(by='Accuracy', ascending=False, inplace=True)
    #reg_lp.sort_values(by='RMSE', inplace=True)
    pct_lp = clf_lp*10
    print(pct_lp)

In [ ]:
#as one big script to go through n_games
n_games = [1,2,3,4,5,6,7,10]
for n in n_games:
    run_all_lp(n)

# Deep Learningggg

In [ ]:
def build_DNN_model(x_len):
    model = Sequential()
    model.add(Dense(63, input_dim = x_len))
    model.add(Activation('relu'))
    model.add(Dropout(0.03))
    model.add(BatchNormalization())

    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.02))

    model.add(Dense(16))
    model.add(Activation('relu'))

    model.add(Dense(8))
    model.add(Activation('relu'))
    #add output layer
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)

    model.compile(loss="mean_squared_error", optimizer=opt, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    print(model.summary())
    return model

In [ ]:
def build_CNN_model(x_len):
    #del model
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=14,
                     input_shape=(x_len, 1)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters=16, kernel_size=10,
                     input_shape=(32, 1)))
    model.add(Activation('linear'))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=10, kernel_size=8,
                     input_shape=(16, 1)))
    model.add(Activation('linear'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(64, activation='linear'))
    model.add(Dense(32, activation='linear'))
    model.add(Dense(16, activation='linear'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"


    model.compile(loss="mean_squared_error", optimizer=opt, metrics=[tf.keras.metrics.RootMeanSquaredError()])

    print(model.summary())
    return model

In [ ]:
#flag = 0 (DNN)
#flag = 1 (CNN)
def eval_dl(x,y,k,flag):
    cv = StratifiedKFold(n_splits=k,shuffle=True)
    best_model = []
    results = []
    highest = 0
    i = 1
    for train,test in cv.split(x,y):
        if(flag == 0):
            model = build_DNN_model(x[train].shape[1])
        if(flag == 1):
            x = x.reshape(x.shape[0], x.shape[1], 1)
            model = build_CNN_model(x[train].shape[1])
        bs = ((x[train].shape[0])/20)
        bs = round(bs)
        history = model.fit(x[train], y[train], validation_data=(x[test], y[test]), epochs = 50, batch_size=bs)
        _, accuracy = model.evaluate(x[test], y[test], batch_size=bs, verbose=0)
        accuracy = accuracy * 100
        print("accuracy for model " + str(i) + " is " + str(accuracy))
        if(accuracy > highest):
            highest = accuracy
            best_model = model
        results.append(accuracy)
        i = i + 1
    print("highest accuracy is: " + str(highest))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_model